In [ ]:
import tensorflow as tf  # version 0.12.0rc0
import rcnn

# %config InlineBackend.figure_format = 'retina'  # double res graphs

# Load constrained dataset
from grid_dataset import load_grid_dataset
datasets = load_grid_dataset('data/grids_9x9_1000.hdf5')

# Some magic to autoreload external imports
# from http://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [ ]:
# Hyperparameters
learning_rate = 1e-3
training_iters = 2000000
batch_size = 128
display_step = 50  # Steps after to give some feedback during training
validation_step = 1000  # Steps after to run a round of validation to 
                        # check against overfitting

# Network Parameters
n_input = 1000 # Data input
n_classes = 2 # Connected or not connected
grid_size = 9
height = grid_size
width = grid_size
depth = 1
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, shape=[None, height, width, depth])
y = tf.placeholder(tf.int32, shape=[None])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [ ]:
# Construct model
rcnn = rcnn.Model(X, y, output_size=2, learning_rate=learning_rate, dropout=keep_prob)

# Initializing the variables
# Note: tensorflow needs to have this initializer after
# the model constructor above in order to catch all variables 
init = tf.global_variables_initializer()

# Merge all summaries for tensorboard writer 
# merged = tf.summary.merge_all()

# Add a summary writer for train steps
# train_writer = tf.summary.FileWriter("/tmp/tensorboard", graph=tf.get_default_graph())

In [5]:
## Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        
        batch_X, batch_y = datasets.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(rcnn.train, feed_dict={X: batch_X, y: batch_y,
                                        keep_prob: dropout})
        
        # Display training steps
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([rcnn.loss, rcnn.predict], 
                                 feed_dict={X: batch_X,
                                            y: batch_y,
                                            keep_prob: dropout})
#             train_writer.add_summary(summary, step)
            
            print("Iteration " + str(step * batch_size) + \
                  ", Minibatch Loss= " + "{:.6f}".format(loss) + \
                  ", Training Accuracy= " + "{:.5f}".format(acc))
        
        # Run a round of validation once in a while
        if step % validation_step == 0:
            print("Validation Accuracy:", \
                  sess.run(rcnn.predict, feed_dict={X: datasets.validation.X[:batch_size],
                                                      y: datasets.validation.y[:batch_size],
                                                      keep_prob: 1.}))
            
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy
    print("Testing Accuracy:", \
          sess.run(rcnn.predict, feed_dict={X: datasets.test.X[:batch_size],
                                            y: datasets.test.y[:batch_size],
                                            keep_prob: 1.}))


Iteration 6400, Minibatch Loss= 3310.191895, Training Accuracy= 0.51562
Iteration 12800, Minibatch Loss= 963.414795, Training Accuracy= 0.64844
Iteration 19200, Minibatch Loss= 541.648438, Training Accuracy= 0.66406


KeyboardInterrupt: 